In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

In [3]:
# MobileNetV2 expect a 224*224 image size  
img_size = 224
# Number of images passed at once during training.
batch_size = 32
epochs = 10

# Normalizes pixel values from [0,255] to [0,1]
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_gen = datagen.flow_from_directory(
    "images",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    "images",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 1282 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [ ]:
# load MobileNetV2 pretrained cnn on ImageNet.
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), 
                         include_top=False, # remove the default 1000-class classification layer.
                         weights='imagenet')# Loads the pretrained weights from ImageNet.

#Freezes all layers in base_model so their weights won’t be updated during training.
base_model.trainable = False  

# only training the custom head, not the entire model.
# compute the avg value of each 7×7 feature map (returned by the mobilenet).
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_gen, validation_data=val_gen, epochs=epochs)

In [4]:
model.save('mask_model.h5')